In [1]:
import pandas as pd
from conceptualDesign.conceptualDesign import conceptualDesign
from misc.openData import openData
from misc.materials import load_materials
import matplotlib.pyplot as plt

In [ ]:
material_data: dict = load_materials()

parameters = openData("design1")
params, df = conceptualDesign(parameters, material_data)

    cargoMass  cargoVolume  passengers  passengerMass  payloadMass  altitude  \
0           0            0         500            100        50000      5000   
1           0            0         500            100        50000      5000   
2           0            0         500            100        50000      5000   
3           0            0         500            100        50000      5000   
4           0            0         500            100        50000      5000   
..        ...          ...         ...            ...          ...       ...   
95          0            0         500            100        50000      5000   
96          0            0         500            100        50000      5000   
97          0            0         500            100        50000      5000   
98          0            0         500            100        50000      5000   
99          0            0         500            100        50000      5000   

    velocity  flightRange   shape  comp

In [ ]:
df[["fuelMass", "balloonVolume", "requiredEnergy", "totalMass"]]

In [ ]:
plt.plot(df.index, df.totalMass, df.fuelMass)